In [ ]:
import pandas as pd
from pprint import pprint
import pandas as pd

from requests import Session
session = Session()
session.verify = False

In [ ]:
# Enable Tenable Log
import logging
logging.basicConfig(level=logging.DEBUG)

In [ ]:
from tenable.io import TenableIO

from tenable_authen import TenableAuthentication

credential = TenableAuthentication().getCredential()

TIO_ACCESS_KEY = credential["TENABLE_ACCESSKEY"]
TIO_SECRET_KEY = credential["TENABLE_SECRETKEY"]

PROXIES_HTTPS =  TenableAuthentication().getProxies()

PROXIES = {
    "https": PROXIES_HTTPS
}

tenableIO = TenableIO(TIO_ACCESS_KEY, TIO_SECRET_KEY, session=session, proxies=PROXIES)

# del credential, TIO_ACCESS_KEY, TIO_SECRET_KEY

## Retrieve the list of scans

In [ ]:
def getlistOfScans():
    return tenableIO.scans.list()

In [ ]:
pd.DataFrame(getlistOfScans())

## Retrieve the Scans Detail

In [ ]:
def getScanDetail(scan_id=""):
    return tenableIO.scans.details(scan_id=scan_id)

In [ ]:
scan_id=73

In [ ]:
getScanDetail(scan_id=scan_id)

## Retrieve the list of scan histories

In [ ]:
def getlistOfScanhistories(scan_id=""):
    return tenableIO.scans.history(scan_id=scan_id)

In [ ]:
scan_id=73

In [ ]:
pd.DataFrame(getlistOfScanhistories(scan_id=scan_id))

## Retrieve the Scans Status Information

In [ ]:
def getScanStatusInformation(scan_id="", history_uuid=""):
    return tenableIO.scans.info(scan_id=scan_id, history_uuid=history_uuid )

In [ ]:
scan_id=73
history_uuid="1cc4791c-f21c-47ac-b96c-85xxxxxxxf42"

In [ ]:
getScanStatusInformation(scan_id=scan_id, history_uuid=history_uuid)

## Create a New Scan

https://developer.tenable.com/docs/determine-settings-for-credential-type

In [ ]:
def createNewScan(name="", targets=[], credentials=None, scanner=None, template=None, compliance=None):
    return tenableIO.scans.create(name=name,
                                  targets=targets,
                                  credentials=credentials,
                                  scanner=scanner,
                                  template=template,
                                  compliance=compliance
                                 )

In [ ]:
name='Example Scan'
targets=['127.0.0.1']

credentials={'Host': {'SSH': [{'id': 'CREDENTIAL-UUID'}]}}
# credentials={'Host': {'Windows': [{'domain': '', 'username': 'Administrator', 'password': 'sekretsquirrel', 'auth_method': 'Password'}]}}

scanner = ""

template = None
compliance = {}


In [ ]:
createNewScan(name=name, targets=targets, credentials=credentials, scanner=scanner, template=template, compliance=compliance)

## ManageStatus Scan by ScanId

In [ ]:
def startScan(scan_id=""):
    return tenableIO.scans.launch(scan_id=scan_id)

In [ ]:
def stopScan(scan_id=""):
    return tenableIO.scans.stop(scan_id=scan_id)

In [ ]:
def pauseScan(scan_id=""):
    return tenableIO.scans.pause(scan_id=scan_id)

In [ ]:
def resumeScan(scan_id=""):
    return tenableIO.scans.resume(scan_id=scan_id)

In [ ]:
def deleteScan(scan_id=""):
    return tenableIO.scans.delete(scan_id=scan_id)

## Get Status for latest Instance Scan

In [ ]:
def getStatusforlatestInstanceScan(scan_id=""):
    return tenableIO.scans.status(scan_id=scan_id)

In [ ]:
scan_id=44

In [ ]:
getStatusforlatestInstanceScan(scan_id=scan_id)

## Retrieve the scan results from either the latest scan or a specific scan instance in the history.

In [ ]:
def getScansResult(scan_id="", history_id=None, history_uuid=None):
    return tenableIO.scans.results(scan_id=scan_id, history_id=history_id, history_uuid=history_uuid)

In [ ]:
scan_id=165
history_id=None
history_uuid=None

In [ ]:
scanResult = getScansResult(scan_id=scan_id, history_id=history_id, history_uuid=history_uuid)
scanResult.keys()

In [ ]:
scanResult["info"]

In [ ]:
pd.DataFrame(scanResult["hosts"])

In [ ]:
scanResult_df = pd.DataFrame(scanResult["vulnerabilities"])
scanResult_df.sort_values("plugin_id")
# scanResult_df[scanResult_df["severity"]>0]

In [ ]:
pd.DataFrame(scanResult["comphosts"])

In [ ]:
compliance = pd.DataFrame(scanResult["compliance"])
compliance#[["plugin_name"]]

# compliance[["plugin_name"]].to_csv("compliance.csv", index=False)

In [ ]:
scanResult["compliance"]

In [ ]:
pd.DataFrame(scanResult["filters"])

In [ ]:
scanResult["notes"]

In [ ]:
scanResult["remediations"]

## Export Report 

In [ ]:
import requests

url = f"https://cloud.tenable.com/scans/{scan_id}/export"

payload = {"format": "csv"}

headers = {
    "accept": "application/json",
    "content-type": "application/json",
    "X-ApiKeys": f"accessKey={TIO_ACCESS_KEY};secretKey={TIO_SECRET_KEY}"
}

response = requests.post(url, json=payload, headers=headers, proxies=PROXIES)

if response.status_code == 200:
    file_id = response.json()
    file_id = file_id["file"]
    print(file_id)


In [ ]:
url = f"https://cloud.tenable.com/scans/{scan_id}/export/{file_id}/status"

headers = {
    "accept": "application/json",
    "X-ApiKeys": f"accessKey={TIO_ACCESS_KEY};secretKey={TIO_SECRET_KEY}"
}

response = requests.get(url, headers=headers, proxies=PROXIES)

if response.status_code == 200:
    res = response.json()
    print(res["status"])

In [ ]:
url = f"https://cloud.tenable.com/scans/{scan_id}/export/{file_id}/download"

headers = {
    "accept": "application/octet-stream",
    "X-ApiKeys": f"accessKey={TIO_ACCESS_KEY};secretKey={TIO_SECRET_KEY}"
}

response = requests.get(url, headers=headers, proxies=PROXIES)
res = response.text

In [ ]:
from io import StringIO

pd.read_csv(StringIO(res))

## Export the scan report.

In [ ]:
def exportScansReport(scan_id="", history_id=None):
    with open('outputs/ScansReport.csv', 'wb') as reportobj:
        tenableIO.scans.export(scan_id=scan_id, history_id=history_id, format="csv", fobj=reportobj)
        

In [ ]:
scan_id=165
history_id=None

In [ ]:
exportScansReport(scan_id=scan_id, history_id=history_id)